In [47]:
!pip install -U langchain-community
!pip install unstructured
!pip install openai
!pip install tiktoken
!pip install chromadb
!pip install tenacity
!pip install --upgrade langchain
!pip install pypdf

In [68]:
!pip install update langchain  langchain-core -qU langchain-openai langchain_community langchain_chroma

In [59]:
!pip install langchain_community langchain_chroma
!pip install -qU langchain-openai
!pip install langchain
!pip install langchain-core

  Using cached langchain-0.2.12-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.2.29-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain-0.2.12-py3-none-any.whl (990 kB)
Using cached langchain_core-0.2.29-py3-none-any.whl (383 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.5
    Uninstalling langchain-core-0.2.5:
      Successfully uninstalled langchain-core-0.2.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.3
    Uninstalling langchain-0.2.3:
      Successfully uninstalled langchain-0.2.3


In [69]:
!pip install langchain langchain-core --upgrade

In [53]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [43]:
import os
os.environ["OPENAI_API_KEY"] = "key"

In [36]:
from langchain.document_loaders import  DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import shutil
import chromadb


DATA_PATH = "/content/sample_data"

def load_documents():
  loader = DirectoryLoader(DATA_PATH,glob="*.md")
  documents = loader.load()
  return documents

documents=load_documents()



In [23]:
#recursive character text splitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True,
)

chunks=text_splitter.split_documents(documents)

In [27]:
print(f"Split {len(documents)} document into {len(chunks)} chunks ")
document=chunks[100]
print(document.page_content)
print(document.metadata)

Split 1 document into 285 chunks 
It was, no doubt: only Alice did not like to be told so. “It’s really dreadful,” she muttered to herself, “the way all the creatures argue. It’s enough to drive one crazy!”

The Footman seemed to think this a good opportunity for repeating his remark, with variations. “I shall sit here,” he said, “on and off, for days and days.”

“But what am I to do?” said Alice.

“Anything you like,” said the Footman, and began whistling.

“Oh, there’s no use in talking to him,” said Alice desperately: “he’s perfectly idiotic!” And she opened the door and went in.

The door led right into a large kitchen, which was full of smoke from one end to the other: the Duchess was sitting on a three-legged stool in the middle, nursing a baby; the cook was leaning over the fire, stirring a large cauldron which seemed to be full of soup.

“There’s certainly too much pepper in that soup!” Alice said to herself, as well as she could for sneezing.
{'source': '/content/sample_data/a

In [74]:
CHROMA_PATH = "chroma"

# Initialize a rate limit handler
rate_limit_handler = CustomRateLimitCallbackHandler(max_calls_per_minute=5)

# Pass the handler to the embedding function
db = Chroma.from_documents(
    chunks,
    OpenAIEmbeddings(callbacks=[rate_limit_handler]),  # Add the handler here
    persist_directory=CHROMA_PATH
)

# making sure the db stays in the computer locally
db.persist()
print(f"saved {len(chunks)} chunks to the {CHROMA_PATH}.")

/usr/local/lib/python3.10/dist-packages/langchain_community/embeddings/openai.py:272: UserWarning: WARNING! callbacks is not default parameter.
                    callbacks was transferred to model_kwargs.
                    Please confirm that callbacks is what you intended.
  warnings.warn(


TypeError: Embeddings.create() got an unexpected keyword argument 'callbacks'

In [82]:
from google.colab import userdata
userdata.get('HF_TOKEN')
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode()



TypeError: SentenceTransformer.encode() missing 1 required positional argument: 'sentences'

In [77]:
CHROMA_PATH = "chroma"
embeddings = model.encode()

db = Chroma.from_documents(
    chunks,
    embeddings,
    persist_directory=CHROMA_PATH
)

# making sure the db stays in the computer locally
db.persist()
print(f"saved {len(chunks)} chunks to the {CHROMA_PATH}.")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [84]:
from google.colab import userdata
userdata.get('HF_TOKEN')
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
#from chromadb import Chroma

# Load the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


# Generate embeddings for the chunks
embeddings = model.encode(chunks)

# Specify the directory where the Chroma database will be stored
CHROMA_PATH = "chroma"

# Create the Chroma database with the generated embeddings
db = Chroma.from_documents(
    chunks,  # The original chunks (documents)
    embeddings,  # The embeddings generated for those chunks
    persist_directory=CHROMA_PATH  # Directory to store the database
)

# Persist the database locally
db.persist()
print(f"Saved {len(chunks)} chunks to the {CHROMA_PATH}.")


TypeError: 'Document' object is not subscriptable

In [85]:
from google.colab import userdata
userdata.get('HF_TOKEN')
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
#from chromadb import Chroma

# Load the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Extract the text content from the Document objects
# The encode method expects a list of strings
chunk_texts = [chunk.page_content for chunk in chunks]

# Generate embeddings for the chunks
embeddings = model.encode(chunk_texts)

# Specify the directory where the Chroma database will be stored
CHROMA_PATH = "chroma"

# Create the Chroma database with the generated embeddings
db = Chroma.from_documents(
    chunk_texts,  # The text
    embeddings,  # The embeddings generated for those chunks
    persist_directory=CHROMA_PATH  # Directory to store the database
)

# Persist the database locally
db.persist()
print(f"Saved {len(chunks)} chunks to the {CHROMA_PATH}.")

AttributeError: 'numpy.ndarray' object has no attribute 'embed_documents'

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
#from chromadb import Chroma

# Load the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Extract the text content from the Document objects
# The encode method expects a list of strings
chunk_texts = [chunk.page_content for chunk in chunks]

# Generate embeddings for the chunks
embeddings = model.encode(chunk_texts)

# Specify the directory where the Chroma database will be stored
CHROMA_PATH = "chroma"

# Create the Chroma database with the generated embeddings
# Pass the model itself as the embedding function
db = Chroma.from_documents(
    chunks,  # The text
    model,  # Pass the SentenceTransformer model here
    persist_directory=CHROMA_PATH  # Directory to store the database
)

# Persist the database locally
db.persist()
print(f"Saved {len(chunks)} chunks to the {CHROMA_PATH}.")

In [35]:
# clear out the previous chroma database path
if os.path.exists(CHROMA_PATH):
  shutil.rmtree(CHROMA_PATH)

In [78]:
# Sentence Transformers
!pip install -U sentence-transformers


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu